In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import os 



N=2000
M=5000
init_infected=0.01
prob_inf=0.05
prob_rec=0.05
prob_death=0.01
small_m=2#the number of attached edges from a new added node in barabasi albert graph generating scheme

def initialize():
    global g, nextg, pos_fix, ever_infected, total_deceased
    #g = nx.gnm_random_graph(N,M)
    g=nx.barabasi_albert_graph(N,small_m)
    pos_fix = nx.spring_layout(g,k=0.65) #always plot at the same positions
    infected=random.sample(range(N),int(init_infected*N))
    ever_infected= len(infected)
    for i in list(g.nodes()):
        g.nodes[i]["state"] = "susceptible"
    for i in infected:
        g.nodes[i]["state"] = "infected"
    #for i in [123,51,93,1,2,3,5,6,7,8,10,11,12,13]:
        #g.node[i]["state"]="recovered" #maybe implement that some people are immune already & implement dying?
    nextg = g.copy()
    
    
def observe():
    global g, nextg,ever_infected, total_deceased, counter
    plt.close()
    fig= plt.figure(figsize=(25,25)) # draw_networkx_nodes versus nx.draw
    nx.draw(g,  node_color = ["r" if g.nodes[i]["state"]=="infected" 
                   else "g" if g.nodes[i]["state"]=="recovered"  
                   else "black" if g.nodes[i]["state"]=="dead"
                   else "gold" if g.nodes[i]["state"]=="immune"
                   else "b" for i in range(N)],pos = pos_fix,node_size=[22*g.degree(i) for i in range(N)])
    plt.title("Epidemic evolution,step "+str(counter),fontsize=30)
    
def observe_nodes():
    global g, nextg,ever_infected, total_deceased, counter
    plt.close()
    fig= plt.figure(figsize=(25,25)) # draw_networkx_nodes versus nx.draw
    nx.draw_networkx_nodes(g,  node_color = ["r" if g.nodes[i]["state"]=="infected" 
                   else "g" if g.nodes[i]["state"]=="recovered"  
                   else "black" if g.nodes[i]["state"]=="dead"
                   else "gold" if g.nodes[i]["state"]=="immune"
                   else "b" for i in range(N)],pos = pos_fix,node_size=[22*g.degree(i) for i in range(N)])
    plt.title("Epidemic evolution, step "+str(counter),fontsize=30)
    
def update():
    global g, nextg, ever_infected, total_deceased, prob_inf
    nextg=g.copy() #we need to declare the relation between g and nextg!
    for i in list(g.nodes()):
        if (g.nodes[i]["state"]=="infected"):
            if(g.degree(i)>0):
                for j in g.neighbors(i):
                    if (g.nodes[j]["state"]=="susceptible"):
                        if random.random()<prob_inf:
                            nextg.nodes[j]["state"] = "infected" #if random()<prob_inf 
                            ever_infected+=1
            if(random.random()<prob_rec):
                nextg.nodes[i]["state"]="recovered"
            #elif(random.random()<prob_death):
                #nextg.node[i]["state"]="dead"
                #total_deceased+=1
                #nextg.remove_edges_from
                #perhaps in the future, remove_node?
    g, nextg = nextg, g                        
                    
    
#script_dir = os.path.dirname("directory")
#results_dir = os.path.join( "Results1/")
#sample_file_name = "sample1511512"

#if not os.path.isdir(results_dir):
    #os.makedirs(results_dir)

#plt.plot([1,2,3,4])
#plt.ylabel("some numbers")
#plt.savefig(results_dir + sample_file_name)   
        
initialize()
counter=0
infected_nodes = [n for n,v in g.nodes(data=True) if v["state"] == "infected"]  
while(len(infected_nodes)>0):
    print(counter)
    update()
    observe()
    plt.savefig("evolution_barabasi_graph/with_edges/evol_N_equal_"+str(N)+"_and_M_equal"+str(M)+"_"+str(int(counter))+".png")    
    observe_nodes()
    plt.savefig("evolution_barabasi_graph/without_edges/evol_N_equal_"+str(N)+"_and_M_equal"+str(M)+"_"+str(int(counter))+".png")    
    counter+=1
    infected_nodes = [n for n,v in g.nodes(data=True) if v["state"] == "infected"]  
    plt.show()

#repetitions=5
#probabilities=np.linspace(0,0.1,50)    
#pandemic_time=np.zeros(probabilities.size)
#total_population_involved=np.zeros(probabilities.size)
#for i in range(probabilities.size):
 #   print(i)
 #   prob_inf=probabilities[i]
 #   counter=0
 #   for j in range(repetitions):
 #       print(j)
 #       initialize()
 #       infected_nodes=[n for n,v in g.nodes(data=True) if v["state"] == "infected"] 
    #script_dir = os.path.dirname("directory")
    #results_dir = os.path.join("Results_prob_inf_"+str(round(prob_inf,3))+"/")
    #if not os.path.isdir(results_dir):
    #    os.makedirs(results_dir)
 #       while(len(infected_nodes)>0):
 #           update()
 #           #observe()
            #file_name = "evol_prob_"+"_"+str(int(counter))+".png"
            #plt.savefig(results_dir+file_name) 
            #plt.close()
 #           counter+=1
 #           infected_nodes = [n for n,v in g.nodes(data=True) if v["state"] == "infected"]  
        #plt.show()
        
 #       total_population_involved[i]+=ever_infected #this adds contributions from all repetitions
 #   total_population_involved[i]/=repetitions
 #   pandemic_time[i]=counter/repetitions
    

#fig= plt.figure(figsize=(20,20))
#plt.plot(probabilities,pandemic_time,color='r',linewidth=2)
#plt.title("Pandemic time as a function of infection probability,\n averaged over "+str(repetitions)+" repetitions.",fontsize=30)
#plt.yticks(fontsize=14)
#plt.ylabel("Pandemic time",fontsize=30)
#plt.xticks(fontsize=14)
#plt.xlabel("Infection probability",fontsize=30)
#plt.savefig("Pandemic_time_Barabasi,5rep.png")
#plt.show()

#fig2=plt.figure(figsize=(20,20))
#plt.plot(probabilities,100*total_population_involved/N,color='r',linewidth=2)
#plt.title("Population involved as a function of infection probability,\n averaged over "+str(repetitions)+" repetitions.",fontsize=30)
#plt.yticks(fontsize=14)
#plt.ylabel("Population_involved [%]",fontsize=30)
#plt.xticks(fontsize=14)
#plt.xlabel("Infection probability",fontsize=30)
#plt.savefig("Population_involved_Barabasi,5rep.png")
#plt.show()

    
    
          #print (selected_nodes)

      #print(random.random())
  
    #plt.savefig("try.png")
    
    #g.nodes()
    
        #fig= plt.figure(figsize=(25,25))
#nx.draw(g, node_color = ["r" if g.node[i]["state"]=="infected" else "b" for i in range(N)],
#pos = g.pos)
#g=nx.karate_club_graph()  
#positions = nx.spring_layout(g)
#nx.draw(g, positions, with_labels = True)
#plt.title("showing node names")
#plt.show()

#listtest=random.sample(range(100),k=42)
#np.sort(np.asarray(listtest))...

0


1


2


3


4


...

231


232


233


234


235
